# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
# Import all needed Python modules
import os
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import dates

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails
from azureml.core.runconfig import DockerConfiguration
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.sampling import BayesianParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.31.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
# Use current workspace 
ws = Workspace.from_config()

# Use default datastore
datastore = ws.get_default_datastore()

# Choose a name for experiment  
experiment_name = 'capstone-project'
experiment=Experiment(ws, experiment_name)

# Choose a name for your CPU cluster
# If the cluster exists, use it. Otherwise create it
amlcompute_cluster_name = 'capstone-compute'
compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)

In [3]:
%%writefile conda_dependencies.yml

channels:
- conda-forge
dependencies:
- python=3.7
- pip:
  - matplotlib
  - joblib
  - psutil==5.8.0
  - tqdm==4.59.0
  - pandas==1.1.5
  - scipy==1.5.4
  - numpy==1.16.0
  - azureml-core==1.30.0
  - azureml-defaults==1.30.0
  - azureml-telemetry==1.30.0
  - tensorboard==2.4.0
  - tensorflow-gpu==2.4.1
  - horovod[tensorflow-gpu]==0.21.3
  - scikit-learn==0.24.0

Overwriting conda_dependencies.yml


In [4]:
%%writefile dockerfile

FROM mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04:20210615.v1

ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/tensorflow-2.4

# Create conda environment
RUN conda create -p $AZUREML_CONDA_ENVIRONMENT_PATH \
    python=3.7 pip=20.2.4

# Prepend path to AzureML conda environment
ENV PATH $AZUREML_CONDA_ENVIRONMENT_PATH/bin:$PATH

# Install pip dependencies
RUN HOROVOD_WITH_TENSORFLOW=1 \
    pip install 'matplotlib>=3.3,<3.4' \
                'psutil>=5.8,<5.9' \
                'tqdm>=4.59,<4.60' \
                'pandas>=1.1,<1.2' \
                'scipy>=1.5,<1.6' \
                'numpy>=1.10,<1.20' \
                'azureml-core==1.30.0' \
                'azureml-defaults==1.30.0' \
                'azureml-telemetry==1.30.0' \
                'tensorboard==2.4.0' \
                'tensorflow-gpu==2.4.1' \
                'horovod[tensorflow-gpu]==0.21.3' \
                'scikit-learn==0.24.0'

# This is needed for mpi to locate libpython
ENV LD_LIBRARY_PATH $AZUREML_CONDA_ENVIRONMENT_PATH/lib:$LD_LIBRARY_PATH

Overwriting dockerfile


In [5]:
# # If you used the conda file created above...
# env = Environment.from_conda_specification(name = 'tensorflow-keras-sklearn-training', file_path = './conda_dependencies.yml')
# # Specify a GPU base image
# env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.0-cudnn7-ubuntu18.04'
# print(env)

# If you used the dockerfile created above...
env = Environment.from_dockerfile(name = 'tensorflow-keras-sklearn-training', dockerfile = './dockerfile')
print(env)

env.register(workspace=ws)

Environment(Name: tensorflow-keras-sklearn-training,
Version: None)


{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": "\nFROM mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04:20210615.v1\n\nENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/tensorflow-2.4\n\n# Create conda environment\nRUN conda create -p $AZUREML_CONDA_ENVIRONMENT_PATH \\\n    python=3.7 pip=20.2.4\n\n# Prepend path to AzureML conda environment\nENV PATH $AZUREML_CONDA_ENVIRONMENT_PATH/bin:$PATH\n\n# Install pip dependencies\nRUN HOROVOD_WITH_TENSORFLOW=1 \\\n    pip install 'matplotlib>=3.3,<3.4' \\\n                'psutil>=5.8,<5.9' \\\n                'tqdm>=4.59,<4.60' \\\n                'pandas>=1.1,<1.2' \\\n                'scipy>=1.5,<1.6' \\\n                'numpy>=1.10,<1.20' \\\n                'azureml-core==1.30.0' \\\n                'azureml-defaults==1.30.0' \\\

In [6]:
# Unzip Kaggle file and only load in the Ethereum data
with zipfile.ZipFile('./data/cryptocurrencypricehistory.zip') as myzip:
    with myzip.open('coin_Ethereum.csv') as Ethereum:
        ETH = pd.read_csv(Ethereum)   
    with myzip.open('coin_Bitcoin.csv') as Bitcoin:
        BTC = pd.read_csv(Bitcoin)   
       
ETH.drop(['SNo', 'Name', 'Symbol'], inplace=True, axis=1)
BTC.drop(['SNo', 'Name', 'Symbol'], inplace=True, axis=1)

crypto = ETH.merge(BTC, how='inner', on='Date', suffixes=['_ETH', '_BTC'])

# Change datetime column to be a date and sort by it
crypto.loc[:,'Date'] = pd.to_datetime(crypto.Date).dt.date
crypto.set_index('Date', inplace=True, drop=True)
crypto.sort_index(ascending=True, inplace=True)
# Add new columns that prevent data leakage and remove the old ones
crypto.loc[:,'LowPrevDay_BTC'] = crypto.Low_BTC.shift(1)
crypto.loc[:,'LowPrevDay_ETH'] = crypto.Low_ETH.shift(1)
crypto.loc[:,'HighPrevDay_BTC'] = crypto.High_BTC.shift(1)
crypto.loc[:,'HighPrevDay_ETH'] = crypto.High_ETH.shift(1)
crypto.loc[:,'VolumePrevDay_BTC'] = crypto.Volume_BTC.shift(1)
crypto.loc[:,'VolumePrevDay_ETH'] = crypto.Volume_ETH.shift(1)
crypto.loc[:,'MarketcapPrevDay_BTC'] = crypto.Marketcap_BTC.shift(1)
crypto.loc[:,'MarketcapPrevDay_ETH'] = crypto.Marketcap_ETH.shift(1)
crypto.loc[:,'HiLoDiff_ETH'] = crypto.HighPrevDay_ETH - crypto.LowPrevDay_ETH
crypto.loc[:,'HiLoDiff_BTC'] = crypto.HighPrevDay_BTC - crypto.LowPrevDay_BTC
crypto.loc[:,'Close_MA7_ETH'] = crypto.Close_ETH.rolling(7).mean()
crypto.loc[:,'Close_MA14_ETH'] = crypto.Close_ETH.rolling(14).mean()
crypto.loc[:,'Close_STDEV7_ETH'] = crypto.Close_ETH.rolling(7).std()
crypto.loc[:,'Close_STDEV14_ETH'] = crypto.Close_ETH.rolling(14).std()
crypto.drop(['Close_BTC', 'Low_BTC', 'Low_ETH', 'High_BTC', 'High_ETH', 'Volume_BTC', 'Volume_ETH', 'Marketcap_BTC', 'Marketcap_ETH'], axis=1, inplace=True)
crypto.drop(index=crypto.index[0], axis=0, inplace=True) 
crypto.dropna(inplace=True)

crypto.to_csv('./data/crypto.csv')
crypto = pd.read_csv('./data/crypto.csv')
crypto.head()

,Date,Open_ETH,Close_ETH,Open_BTC,LowPrevDay_BTC,LowPrevDay_ETH,HighPrevDay_BTC,HighPrevDay_ETH,VolumePrevDay_BTC,VolumePrevDay_ETH,MarketcapPrevDay_BTC,MarketcapPrevDay_ETH,HiLoDiff_ETH,HiLoDiff_BTC,Close_MA7_ETH,Close_MA14_ETH,Close_STDEV7_ETH,Close_STDEV14_ETH
0,2015-08-21,1.47752,1.39529,235.354996,226.899002,1.24833,237.365005,1.53330,32275000.0,2843760.0,3.417123e+09,1.063514e+08,0.28497,10.466003,1.380666,1.269226,0.211769,0.384875
1,2015-08-22,1.39629,1.37923,232.662003,231.723999,1.35280,236.432007,1.55642,23173800.0,2020970.0,3.377704e+09,1.013319e+08,0.20362,4.708008,1.336427,1.313934,0.163488,0.355575
2,2015-08-23,1.37500,1.35259,230.376007,222.703995,1.35268,234.957001,1.47641,23205900.0,948310.0,3.346962e+09,1.002018e+08,0.12373,12.253006,1.305936,1.360412,0.130004,0.308881
3,2015-08-24,1.34559,1.23127,228.112000,225.580002,1.29777,232.705002,1.40970,18406600.0,1589300.0,3.315467e+09,9.830035e+07,0.11193,7.125000,1.309887,1.397756,0.126755,0.249983
4,2015-08-25,1.22861,1.14019,210.067993,210.442993,1.23127,228.139008,1.36278,59220700.0,924920.0,3.059461e+09,8.951526e+07,0.13151,17.696014,1.317479,1.402923,0.111907,0.243299


In [7]:
# Upload the data to the datastore
datastore.upload_files(files = ['./data/crypto.csv'],
                       target_path = 'crypto/',
                       overwrite = True,
                       show_progress = True)
# See if the data is there 
dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, 'crypto/crypto.csv')])

Uploading an estimated of 1 files
Uploading ./data/crypto.csv
Uploaded ./data/crypto.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [8]:
# Try to get the registered dataset. If its not there, register it
dataset_registered = False
try:
    temp = Dataset.get_by_name(workspace = ws, name = 'crypto')
    dataset_registered = True
except:
    print("The dataset 'crypto' is not registered in workspace yet.")

if not dataset_registered:
    dataset = dataset.register(
        workspace = ws,
        name = 'crypto',
        description='training and test dataset',
        create_new_version=True
        )

# Check to make sure the data is the way you expect it after moving it around from Kaggle to your environment to the datastore to a registered dataset
crypto = dataset.to_pandas_dataframe()
crypto.set_index('Date', inplace=True, drop=True)
crypto.sort_index(ascending=True, inplace=True)
crypto.head()

,Open_ETH,Close_ETH,Open_BTC,LowPrevDay_BTC,LowPrevDay_ETH,HighPrevDay_BTC,HighPrevDay_ETH,VolumePrevDay_BTC,VolumePrevDay_ETH,MarketcapPrevDay_BTC,MarketcapPrevDay_ETH,HiLoDiff_ETH,HiLoDiff_BTC,Close_MA7_ETH,Close_MA14_ETH,Close_STDEV7_ETH,Close_STDEV14_ETH
Date,,,,,,,,,,,,,,,,,
2015-08-21,1.47752,1.39529,235.354996,226.899002,1.24833,237.365005,1.53330,32275000.0,2843760.0,3.417123e+09,1.063514e+08,0.28497,10.466003,1.380666,1.269226,0.211769,0.384875
2015-08-22,1.39629,1.37923,232.662003,231.723999,1.35280,236.432007,1.55642,23173800.0,2020970.0,3.377704e+09,1.013319e+08,0.20362,4.708008,1.336427,1.313934,0.163488,0.355575
2015-08-23,1.37500,1.35259,230.376007,222.703995,1.35268,234.957001,1.47641,23205900.0,948310.0,3.346962e+09,1.002018e+08,0.12373,12.253006,1.305936,1.360412,0.130004,0.308881
2015-08-24,1.34559,1.23127,228.112000,225.580002,1.29777,232.705002,1.40970,18406600.0,1589300.0,3.315467e+09,9.830035e+07,0.11193,7.125000,1.309887,1.397756,0.126755,0.249983
2015-08-25,1.22861,1.14019,210.067993,210.442993,1.23127,228.139008,1.36278,59220700.0,924920.0,3.059461e+09,8.951526e+07,0.13151,17.696014,1.317479,1.402923,0.111907,0.243299


In [9]:
# import argparse
# import json
# import os
# import zipfile
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import MinMaxScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, LSTM
# from tensorflow.keras.metrics import RootMeanSquaredError
# from tensorflow.keras.callbacks import Callback
# from azureml.core import Run
# from azureml.core import Dataset
# from azureml.core.workspace import Workspace

# def dnn_prep(df):
#     target_col = ['Close_ETH']
#     df = df[[c for c in df if c not in target_col] + target_col]
#     values = df.values.astype('float32')
#     scaler = MinMaxScaler(feature_range=(0, 1))
#     scaled = scaler.fit_transform(values)
#     X, y = scaled[:100, :-1], scaled[:100, -1] ########## REMOVE 100s AFTER TESTING ############
#     X = X.reshape(X.shape[0], 1, X.shape[1])
#     return X, y, scaler


# # -------------------------------------------------------------------

# print(crypto.head())

# train_size = int(round(crypto.shape[0]*0.8, 0))
# val_size = int(round(crypto.shape[0]*0.1, 0))
# train_df = crypto.iloc[0:train_size,:]
# val_df = crypto.iloc[train_size:(train_size + val_size),:]
# test_df = crypto.iloc[(train_size + val_size):crypto.shape[0],:]
# print(train_df.shape, val_df.shape, test_df.shape)
# X_train, y_train, scaler_train = dnn_prep(train_df)
# X_val, y_val, scaler_val = dnn_prep(val_df)


# # Build an LSTM model
# model = Sequential()
# model.add(LSTM(32, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Dropout(0.1))
# model.add(LSTM(16, return_sequences=False))
# model.add(Dropout(0.1))
# model.add(Dense(1))

# # Compile the model
# model.compile(optimizer='adam', loss='mse', metrics=[RootMeanSquaredError(), 'mae', 'mape'])

# epochs=10

# history = model.fit(
#     X_train, y_train,
#     epochs=epochs,
#     verbose=0,
#     validation_data=(X_val, y_val)
# )

# print(history)

# score = model.evaluate(X_val, y_val, verbose=0)
# rmse = score[0]
# print('Root Mean Squared Error:', rmse)
# mae = score[1]
# print('Mean Absolute Error:', mae)
# mape = score[2]
# print('Mean Absolute Percentage Error:', mape)

# plt.figure(figsize=(6, 3))
# plt.title('ETH prediction ({} epochs)'.format(epochs), fontsize=14)
# plt.plot(history.history['root_mean_squared_error'], 'b--', label='Train RMSE', lw=4, alpha=0.5)
# plt.plot(history.history['val_root_mean_squared_error'], 'r--', label='Val RMSE', lw=4, alpha=0.5)
# plt.legend(fontsize=12)
# plt.grid(True)
# plt.show()

# history.history

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

Its important to make note of the version of keras we are training with so we can use the same version to load it during inference. This can be found in the outputs of the training runs

In [10]:
# Create the different params that you will be using during training
# "Note that only choice, quniform, and uniform are supported for Bayesian optimization" https://docs.microsoft.com/en-us/python/api/azureml-train-core/azureml.train.hyperdrive.bayesianparametersampling?view=azure-ml-py#parameters
param_sampling = BayesianParameterSampling(
    {
        'hidden': choice(32, 64, 128),
        'learning_rate': uniform(0.001, 0.1),
        'dropout': uniform(0, 0.25)
    }
)

# Create an estimator by running crypto_train.py
args = ['--input_data', dataset.as_named_input('crypto')]
docker_config = DockerConfiguration(use_docker=True)
src = ScriptRunConfig(
    source_directory='.', 
    script='crypto_train.py', 
    arguments=args,
    compute_target=compute_target,
    environment=env,
    docker_runtime_config=docker_config
)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_run_config = HyperDriveConfig(
    run_config=src, 
    hyperparameter_sampling=param_sampling, 
    primary_metric_name='Root Mean Squared Error', 
    primary_metric_goal=PrimaryMetricGoal.MINIMIZE, 
    max_total_runs=3
) 

For best results with Bayesian Sampling we recommend using a maximum number of runs greater than or equal to 20 times the number of hyperparameters being tuned. Recommendend value:60.


In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = experiment.submit(hyperdrive_run_config)
print("Run submitted for execution.")

Run submitted for execution.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_b8ab1926-a58a-4f61-ac15-0b7a0c9d3aa6
Web View: https://ml.azure.com/runs/HD_b8ab1926-a58a-4f61-ac15-0b7a0c9d3aa6?wsid=/subscriptions/ac401033-05b1-43d5-a5ea-e5dcb9c75b49/resourcegroups/rg-devtest-databricks-01/workspaces/ml-devtest&tid=fa1a69b7-9b39-4cb7-8701-41985a92e9bb

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-08-12T16:54:26.355248][API][INFO]Experiment created<END>\n""<START>[2021-08-12T16:54:26.845787][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space<END>\n""<START>[2021-08-12T16:54:27.004184][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_b8ab1926-a58a-4f61-ac15-0b7a0c9d3aa6
Web View: https://ml.azure.com/runs/HD_b8ab1926-a58a-4f61-ac15-0b7a0c9d3aa6?wsid=/subscriptions/ac401033-05b1-43d5-a5ea-e5dcb9c75b49/resourcegroups/rg-devtest-databricks-01/workspaces/ml-devtest&tid=fa1a69b7-9b39-4cb7-8701-41985a92e9bb



{'runId': 'HD_b8ab1926-a58a-4f61-ac15-0b7a0c9d3aa6',
 'target': 'capstone-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-08-12T16:54:25.993769Z',
 'endTimeUtc': '2021-08-12T17:23:29.609856Z',
 'properties': {'primary_metric_config': '{"name": "Root Mean Squared Error", "goal": "minimize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '9d56e277-31a7-4c4a-bb7a-408d302bae05',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1051-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.31.0',
  'score': '0.00032465814729221165',
  'best_child_run_id': 'HD_b8ab1926-a58a-4f61-ac15-0b7a0c9d3aa6_0',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://filesharedevacme.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_b8ab1926-a58a-4f61-ac15-0b7a0c9d3aa6/azureml-logs/hyperdr

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [13]:
# Get your best run and save the model from that run.
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_hyperdrive_run.get_metrics())
print(best_hyperdrive_run.get_details())
print(best_hyperdrive_run.get_file_names())

{'Hidden Layers': 64, 'Learning Rate': 0.07660676721573413, 'Dropout': 0.2278440588743216, 'Loss': [0.001230738591402769, 0.0006518422742374241, 0.00033820437965914607, 0.00041763816261664033, 0.00043919330346398056, 0.00021037731494288892, 0.00019562695524655282, 0.00015243361121974885, 0.0001557858195155859, 0.00024802624830044806, 0.00024056479742284864, 0.0005595794063992798, 0.00041389241232536733, 0.00013434875290840864, 0.0003993867721874267, 0.00013915121962781996, 0.00013301461876835674, 0.00018744953558780253, 0.0003734350320883095, 0.00014216764247976243, 0.00018310117593500763, 0.0003222002414986491, 0.000132048488012515, 0.00016786168271210045, 0.00032465814729221165], 'Root Mean Squared Error': 0.00032465814729221165, 'Mean Absolute Error': 0.018018271774053574, 'Mean Absolute Percentage Error': 0.012798110023140907, 'ETH_keras': 'aml://artifactId/ExperimentRun/dcid.HD_b8ab1926-a58a-4f61-ac15-0b7a0c9d3aa6_0/ETH_keras_1628788924.png'}
{'runId': 'HD_b8ab1926-a58a-4f61-ac15-

In [19]:
# # create a model folder in the current directory
# os.makedirs('./hyperdrive_model', exist_ok=True)

# best_hyperdrive_run.download_files(output_directory='./hyperdrive_model')

# import tensorflow
# print('Tensorflow version', tensorflow.__version__)
# from tensorflow.keras.models import load_model
# import joblib

# model = load_model('hyperdrive_model/outputs/ETH_hyperdrive_model') # this doesn't work because the single node compute has a different version of tensorflow (v2.1) than the cluster compute (v2.4)

# # make prediction
# X_train = np.load('hyperdrive_model/X_train.npy')
# y_train = np.load('hyperdrive_model/y_train.npy')
# X_test = np.load('hyperdrive_model/X_test.npy')
# y_test = np.load('hyperdrive_model/y_test.npy')
# # joblib.load('hyperdrive_model/scaler.joblib') # not yet needed
# y_hat = model.predict(X_test)

# model.evaluate(X_train, y_train)
# model.evaluate(X_test, y_test)

In [21]:
# Save the best model
best_hyperdrive_run.register_model(model_name = 'hyperdrive_Ethereum_price_forecast', model_path='./outputs')

Model(workspace=Workspace.create(name='ml-devtest', subscription_id='ac401033-05b1-43d5-a5ea-e5dcb9c75b49', resource_group='rg-devtest-databricks-01'), name=hyperdrive_Ethereum_price_forecast, id=hyperdrive_Ethereum_price_forecast:4, version=4, tags={}, properties={})